In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays


In [6]:
train_df=pd.read_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Final-pipeline\data\1_interim\1_cleaned_train.csv')

,site_id,building_id,primary_use,square_feet,year_built,floor_count,meter,meter_reading,air_temperature,cloud_coverage,...,sea_level_pressure,wind_direction,wind_speed,hour,dayofweek,month,dayofyear,day,year,timestamp
0,1,151,Office,30495,1997.0,8.0,0,21.3000,3.8,0.084656,...,1020.900000,240.0,3.1,4,6,4,101,10,2016,2016-04-10 04:00:00
1,1,151,Office,30495,1997.0,8.0,0,21.3000,3.7,0.000000,...,1021.600000,230.0,2.6,4,6,4,101,10,2016,2016-04-10 04:00:00
2,1,151,Office,30495,1997.0,8.0,0,21.3000,2.6,0.000000,...,1021.900000,0.0,0.0,4,6,4,101,10,2016,2016-04-10 04:00:00
3,1,151,Office,30495,1997.0,8.0,0,21.3000,2.0,0.000000,...,1022.300000,170.0,1.5,4,6,4,101,10,2016,2016-04-10 04:00:00
4,1,151,Office,30495,1997.0,8.0,0,21.3000,2.3,0.000000,...,1022.700000,110.0,1.5,4,6,4,101,10,2016,2016-04-10 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262300,15,1442,Public services,99541,1993.0,NaN,2,66.4117,3.0,1.109988,...,1016.980579,180.0,5.7,4,4,1,1,1,2016,2016-01-01 04:00:00
262301,15,1442,Public services,99541,1993.0,NaN,2,66.4117,2.8,2.000000,...,1007.400000,180.0,7.7,4,4,1,1,1,2016,2016-01-01 04:00:00
262302,15,1442,Public services,99541,1993.0,NaN,2,66.4117,2.8,1.109988,...,1007.500000,180.0,5.1,4,4,1,1,1,2016,2016-01-01 04:00:00
262303,15,1442,Public services,99541,1993.0,NaN,2,66.4117,2.2,1.109988,...,1008.000000,170.0,4.6,4,4,1,1,1,2016,2016-01-01 04:00:00


# Features

- Holiday

In [ ]:

location_df= pd.DataFrame()
location_df['site_id']= np.arange(0,16)
location_df['city']= ['Orlando', 'Heathrow', 'Tempe', 'Washington', 'Berkeley', 'Southampton', 'Washington', 'Ottowa', 'Orlando', 'Austin', 'Saltlake',\
                  'Ottowa', 'Dublin', 'Minneapolis', 'Philadelphia', 'Rochestor']

location_df['country']= ['US', 'UK', 'US', 'US', 'US', 'UK', 'US', 'Canada', 'US', 'US', 'US', 'Canada', 'Ireland', 'US', 'US', 'US']
train_df=train_df.merge(location_df, on='site_id', how='left')

UK=[]
US=[]
CA=[]
IRE=[]

for ptr in holidays.UnitedKingdom(years = 2016).keys():
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years = 2017).keys():
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years = 2018).keys():
    UK.append(str(ptr))
UK.append('2019-01-01')


for ptr in holidays.UnitedStates(years = 2016).keys():
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years = 2017).keys():
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years = 2018).keys():
    US.append(str(ptr))
US.append('2019-01-01')


for ptr in holidays.Canada(years = 2016).keys():
    CA.append(str(ptr))
for ptr in holidays.Canada(years = 2017).keys():
    CA.append(str(ptr))
for ptr in holidays.Canada(years = 2018).keys():
    CA.append(str(ptr))
CA.append('2019-01-01')


for ptr in holidays.Ireland(years = 2016).keys():
    IRE.append(str(ptr))
for ptr in holidays.Ireland(years = 2017).keys():
    IRE.append(str(ptr))
for ptr in holidays.Ireland(years = 2018).keys():
    IRE.append(str(ptr))
IRE.append('2019-01-01')

def holiday_filler(df):
  df['isHoliday']=[0]*(df.shape[0])
  df.loc[df['country']=='US', 'isHoliday']= (df['timestamp'].dt.date.astype('str').isin(US)).astype('int')
  df.loc[df['country']=='UK', 'isHoliday']= (df['timestamp'].dt.date.astype('str').isin(UK)).astype('int')
  df.loc[df['country']=='Canada', 'isHoliday']= (df['timestamp'].dt.date.astype('str').isin(CA)).astype('int')
  df.loc[df['country']=='Ireland', 'isHoliday']= (df['timestamp'].dt.date.astype('str').isin(IRE)).astype('int')

  return df

- Season

In [7]:
# Convert 'timestamp' column to datetime format
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])

# Create the 'season' column based on the 'timestamp' column
train_df['season'] = train_df['timestamp'].apply(lambda x: 'Spring' if x.month in [3, 4, 5]
                                                  else 'Summer' if x.month in [6, 7, 8]
                                                  else 'Autumn' if x.month in [9, 10, 11]
                                                  else 'Winter')

- isDaytime

In [8]:
train_df['IsDayTime']= train_df['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)

- relative humidity

In [9]:
train_df['relative_humidity']= 100*((np.exp((17.67*train_df['dew_temperature'])/
                                            (243.5+train_df['dew_temperature'])))/(np.exp((17.67*train_df['air_temperature'])/
                                                                                          (243.5+train_df['air_temperature']))))


In [ ]:
#Output
train_df.to_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Final-pipeline\data\2_processed\2_fe_train.csv', index=False)


# Test_dataset features

In [ ]:
test_df=pd.read_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Final-pipeline\data\1_interim\1_cleaned_test.csv')

In [ ]:
#adding features in test data
test_df['season']= test_df['timestamp'].apply(lambda x: 'Spring' if x.month==3 or x.month==4 or x.month==5 else 'Summer' if x.month==6 or x.month==7 or x.month==8 
                                                else 'Autumn' if x.month==9 or x.month==10 or 
                                                x.month==11 else 'Winter')
test_df['IsDayTime']= test_df['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)
test_df['relative_humidity']= 100*((np.exp((17.67*test_df['dew_temperature'])/(243.5+test_df['dew_temperature'])))/(np.exp((17.67*test_df['air_temperature'])/(243.5+test_df['air_temperature']))))


In [ ]:
test_df=test_df.merge(location_df, on='site_id', how='left')


In [ ]:
test_df= holiday_filler(test_df)


In [ ]:
test_df['square_feet']=np.log1p(test_df['square_feet'])
test_df.drop(['city','country'], axis=1, inplace=True)


In [ ]:
#Output
test_df.to_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Final-pipeline\data\2_processed\2_fe_test.csv', index=False)
